# The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'G:/github/lesson9/data/sample_data/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [5]:
"""
Let's build a small network with two convolutional layers, followed by one fully connected layer.
Convolutional networks are more expensive computationally, 
so we'll limit its depth and number of fully connected nodes.
"""
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  """Input data.
   input : 输入的要做卷积的图片，要求为一个张量，shape为 [ batch, in_height, in_weight, in_channel ]，
   其中batch为图片的数量，in_height 为图片高度，in_weight 为图片宽度，in_channel 为图片的通道数，灰度
   图该值为1，彩色图为3。（也可以用其它值，但是具体含义不是很理解）
  """
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [6]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.499545
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.856652
Minibatch accuracy: 43.8%
Validation accuracy: 29.2%
Minibatch loss at step 100: 0.522452
Minibatch accuracy: 81.2%
Validation accuracy: 67.7%
Minibatch loss at step 150: 0.896857
Minibatch accuracy: 81.2%
Validation accuracy: 73.9%
Minibatch loss at step 200: 1.091586
Minibatch accuracy: 62.5%
Validation accuracy: 74.5%
Minibatch loss at step 250: 1.154553
Minibatch accuracy: 62.5%
Validation accuracy: 77.5%
Minibatch loss at step 300: 0.391969
Minibatch accuracy: 93.8%
Validation accuracy: 77.2%
Minibatch loss at step 350: 1.111692
Minibatch accuracy: 62.5%
Validation accuracy: 78.0%
Minibatch loss at step 400: 0.871404
Minibatch accuracy: 75.0%
Validation accuracy: 79.6%
Minibatch loss at step 450: 0.419640
Minibatch accuracy: 87.5%
Validation accuracy: 79.8%
Minibatch loss at step 500: 0.916143
Minibatch accuracy: 75.0%
Validation accuracy: 79.9%
M

# Problem 1

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (nn.max_pool()) of stride 2 and kernel size 2.

In [9]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data, train=False):
    """    
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
   """
    # 2D convolution, with 'SAME' padding (i.e. the output feature map has
    # the same size as the input). Note that {strides} is a 4D array whose
    # shape matches the data layout: [image index, y, x, depth].
    conv = tf.nn.conv2d(data,
                        layer1_weights,
                        strides=[1, 1, 1, 1],
                        padding='SAME')
    # Bias and rectified linear non-linearity.
    relu = tf.nn.relu(tf.nn.bias_add(conv, layer1_biases))
    # Max pooling. The kernel size spec {ksize} also follows the layout of
    # the data. Here we have a pooling window of 2, and a stride of 2.
    pool = tf.nn.max_pool(relu,
                          ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1],
                          padding='SAME')
    conv = tf.nn.conv2d(pool,
                        layer2_weights,
                        strides=[1, 1, 1, 1],
                        padding='SAME')
    relu = tf.nn.relu(tf.nn.bias_add(conv, layer2_biases))
    pool = tf.nn.max_pool(relu,
                          ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1],
                          padding='SAME')
    # Reshape the feature map cuboid into a 2D matrix to feed it to the
    # fully connected layers.
    pool_shape = pool.get_shape().as_list()
    reshape = tf.reshape(
        pool,
        [pool_shape[0], pool_shape[1] * pool_shape[2] * pool_shape[3]])
    # Fully connected layer. Note that the '+' operation automatically
    # broadcasts the biases.
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    # Add a 50% dropout during training only. Dropout also scales
    # activations such that no rescaling is needed at evaluation time.
    if train:
      hidden = tf.nn.dropout(hidden, 0.5, seed=SEED)
    return tf.matmul(hidden, layer4_weights) + layer4_biases

    
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [10]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.337938
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.596911
Minibatch accuracy: 56.2%
Validation accuracy: 56.2%
Minibatch loss at step 100: 0.608157
Minibatch accuracy: 75.0%
Validation accuracy: 72.5%
Minibatch loss at step 150: 0.690519
Minibatch accuracy: 87.5%
Validation accuracy: 74.8%
Minibatch loss at step 200: 0.770048
Minibatch accuracy: 75.0%
Validation accuracy: 74.7%
Minibatch loss at step 250: 0.849430
Minibatch accuracy: 75.0%
Validation accuracy: 79.9%
Minibatch loss at step 300: 0.406745
Minibatch accuracy: 87.5%
Validation accuracy: 78.8%
Minibatch loss at step 350: 1.038926
Minibatch accuracy: 75.0%
Validation accuracy: 79.0%
Minibatch loss at step 400: 0.939514
Minibatch accuracy: 75.0%
Validation accuracy: 80.9%
Minibatch loss at step 450: 0.413870
Minibatch accuracy: 87.5%
Validation accuracy: 79.9%
Minibatch loss at step 500: 0.678123
Minibatch accuracy: 81.2%
Validation accuracy: 80.8%
Mi

# Problem 2

Try to get the best performance you can using a convolutional net. Look for example at the classic LeNet5 architecture, adding Dropout, and/or adding learning rate decay.

In [23]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf 
import numpy as np
import sys
import os 
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
logs_train_dir = 'G:/lessons/lesson12/model' 
def weight_variable(shape):   
    # 产生正态分布，标准差为0.1    
    initial = tf.truncated_normal(shape, stddev=0.1)    
    return tf.Variable(initial)  
def bias_variable(shape):    # 创建一个结构为shape矩阵也可以说是数组shape声明其行列，初始化所有值为0.1    
    initial = tf.constant(0.1, shape=shape)    
    return tf.Variable(initial)  
def conv2d(x,W):    
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')  
def max_pool_2x2(x):    
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')  
def inference(images,drop = False):    
    # 第1层卷积    
    with tf.variable_scope('conv1'):        
        W_conv1 = tf.Variable(weight_variable([5, 5, 1, 6]), name="weight")        
        b_conv1 = tf.Variable(bias_variable([6]), name="bias")        
        h_conv1 = tf.nn.relu(conv2d(images, W_conv1) + b_conv1)    
        # print(np.shape(h_conv1))   
    # 第2层池化    
    with tf.variable_scope('pool2'):        
        h_pool2 = max_pool_2x2(h_conv1)    
        # print(np.shape(h_pool2))     
    # 第3层卷积    
    with tf.variable_scope('conv3'):        
         W_conv3 = tf.Variable(weight_variable([5, 5, 6, 16]), name="weight")        
         b_conv3 = tf.Variable(bias_variable([16]), name="bias")        
         h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3) + b_conv3)    
         # print(np.shape(h_conv3))    
    # 第4层池化    
    with tf.variable_scope('pool4'):        
          h_pool4 = max_pool_2x2(h_conv3)    
          # print(np.shape(h_pool4))    
        
    h_pool4_flat = tf.reshape(h_pool4, [-1, 7 * 7 * 16])  
    
    # 5\6\7全连接层    
    with tf.variable_scope('fc5'):        
         W_fc5 = weight_variable([7 * 7 * 16, 120])        
         b_fc5 = bias_variable([120])       
         h_fc5 = tf.nn.relu(tf.matmul(h_pool4_flat, W_fc5) + b_fc5) 
            
    with tf.variable_scope('fc6'):        
         W_fc6 = weight_variable([120, 84])        
         b_fc6 = bias_variable([84])        
         h_fc6 = tf.nn.relu(tf.matmul(h_fc5, W_fc6) + b_fc6)  
         if drop:
             h_fc6  = tf.nn.dropout(h_fc6 , 0.5)   
            
    with tf.variable_scope('out'):        
         W_out = weight_variable([84, 10])        
         b_out = bias_variable([10])        
         h_out = tf.nn.softmax(tf.matmul(h_fc6, W_out) + b_out) 
     
    return h_out 
def train_fun(drop = False):
    x = tf.placeholder(tf.float32, [None, 28*28])
    y_ = tf.placeholder(tf.float32, [None, 10])
    x_image = tf.reshape(x, [-1, 28, 28, 1]) 
    y_conv = inference(x_image,drop) 
    # 定义损失函数和学习步骤
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))   
    # 交叉熵
    train_step = tf.train.AdamOptimizer(1e-3).minimize(cross_entropy)   
    # 用Adam优化器 
    correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))  
    summary_op = tf.summary.merge_all()
    sess = tf.InteractiveSession()
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    train_writer = tf.summary.FileWriter(logs_train_dir, sess.graph) 
    for i in range(2000):    
        batch = mnist.train.next_batch(50)   
        if i % 100 == 0:        
            train_accuracy = accuracy.eval(feed_dict={x: batch[0], y_: batch[1]})        
            print("step %d, training accuracy %g" % (i, train_accuracy))        
            # summary_str = sess.run(summary_op)       
            # train_writer.add_summary(summary_str, i)    
        train_step.run(feed_dict={x: batch[0], y_: batch[1]})
    checkpoint_path = os.path.join(logs_train_dir, 'thing.ckpt')
    saver.save(sess, checkpoint_path)
    print("test accuracy %g" % accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [19]:
train_fun(drop = False)

g:\Anaconda\lib\site-packages\tensorflow\python\client\session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


step 0, training accuracy 0.12
step 100, training accuracy 0.98
step 200, training accuracy 0.94
step 300, training accuracy 0.92
step 400, training accuracy 0.98
step 500, training accuracy 0.96
step 600, training accuracy 0.98
step 700, training accuracy 0.96
step 800, training accuracy 0.98
step 900, training accuracy 0.96
step 1000, training accuracy 1
step 1100, training accuracy 0.98
step 1200, training accuracy 0.98
step 1300, training accuracy 0.98
step 1400, training accuracy 1
step 1500, training accuracy 0.96
step 1600, training accuracy 0.98
step 1700, training accuracy 1
step 1800, training accuracy 0.98
step 1900, training accuracy 1
test accuracy 0.9862


In [24]:
train_fun(drop = True)

g:\Anaconda\lib\site-packages\tensorflow\python\client\session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


step 0, training accuracy 0.06
step 100, training accuracy 0.84
step 200, training accuracy 0.82
step 300, training accuracy 0.88
step 400, training accuracy 0.94
step 500, training accuracy 0.96
step 600, training accuracy 0.96
step 700, training accuracy 0.94
step 800, training accuracy 0.96
step 900, training accuracy 1
step 1000, training accuracy 0.98
step 1100, training accuracy 0.94
step 1200, training accuracy 0.96
step 1300, training accuracy 1
step 1400, training accuracy 0.96
step 1500, training accuracy 0.96
step 1600, training accuracy 1
step 1700, training accuracy 0.96
step 1800, training accuracy 0.96
step 1900, training accuracy 0.94
test accuracy 0.978
